In [8]:
import quimb.tensor as qtn
import numpy as np


In [9]:
N = 10
circ = qtn.Circuit(N)

In [10]:
def brickwork_state_init(qtm_circ):
    for i in [1, 2, 3, 4, 6, 7, 8, 9]:
        qtm_circ.apply_gate('H', i)
    # create the entanglement for the brickwork state
    circ.apply_gate('CZ', 0, 1)
    circ.apply_gate('CZ', 1, 2)
    circ.apply_gate('CZ', 2, 3)
    circ.apply_gate('CZ', 3, 4)
    
    circ.apply_gate('CZ', 5, 6)
    circ.apply_gate('CZ', 6, 7)
    circ.apply_gate('CZ', 7, 8)
    circ.apply_gate('CZ', 8, 9)

    circ.apply_gate('CZ', 2, 7)
    circ.apply_gate('CZ', 4, 9)
    return qtm_circ


In [11]:
circ.apply_gate('X',0)
circ.apply_gate('X',5)
circ = brickwork_state_init(circ)

In [12]:
hadamard_zeroket_zerobra = np.matmul(np.array([[1, 1], [1,-1]]), np.array([[1, 0], [0, 0]]))
hadamard_zeroket_zerobra_hadmard = np.matmul(hadamard_zeroket_zerobra, np.array([[1, 1], [1,-1]]))
proj = 1/2 * hadamard_zeroket_zerobra_hadmard 

def rotation_z_gate(theta):
    rot_z = np.array([[np.exp(-1j*theta), 0],[0, np.exp(1j*theta)]])
    return rot_z

def z_rot_proj_meas(qubit, theta, qtm_circ):
    # change basis 
    #circ.apply_gate('H', qubit)
    z_rot = rotation_z_gate(theta)

    #z_rot_proj = np.matmul(z_rot, np.matmul(proj, z_rot.conjugate()))
    #print(proj)
        
    qtm_circ.apply_gate(z_rot, qubit)
    qtm_circ.apply_gate(proj, qubit)
    print(qubit)
    print(z_rot)
    # renormalize
    state_vector = np.round(qtm_circ.psi.to_qarray(), 10)
    renorm_factor = np.sqrt(np.dot(state_vector.conj().T, state_vector))
    print(renorm_factor)
    renorm_matrix = np.array([[1, 0], [0, 1]])* 1/renorm_factor
    qtm_circ.apply_gate(renorm_matrix, qubit)

In [13]:
z_rot_proj_meas(0, np.pi/4, circ)
z_rot_proj_meas(5, 0, circ)
z_rot_proj_meas(1, np.pi/4, circ)
z_rot_proj_meas(6, 0, circ)
z_rot_proj_meas(2, np.pi/4, circ)
z_rot_proj_meas(7, 0, circ)
for qubit in [3, 8]:
    z_rot_proj_meas(qubit, 0, circ)

0
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]
[[0.707107+0.j]]
5
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[0.707107+0.j]]
1
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]
[[0.707107+0.j]]
6
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[0.707107+0.j]]
2
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]
[[0.707107+0.j]]
7
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[0.707107+0.j]]
3
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[0.707107+0.j]]
8
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[0.707107+0.j]]


In [14]:
#for i in range(10):
    #circ.apply_gate('H', i)
for i in [0, 5, 1, 6, 2, 7, 3, 8]:
    circ.apply_gate('H', i)

list = np.nonzero(np.round(circ.psi.to_qarray(), 10))

sv =np.round(np.exp(-3j * np.pi/2)*circ.psi.to_qarray(), 10)
print(list[0], len(sv))
for i in list[0]:
    print(sv[i])

[ 1 33] 1024
[0.707107+0.j]
[-0.707107+0.j]
